In [1]:
import json
from bs4 import Tag
from tqdm import tqdm
import tbmgar_utils
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [2]:

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()
        return str(obj)

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")

def scrape_article(url, page_key_code):
    try:
        article_content = tbmgar_utils.scrape_tbmgar_article_content(url, tags=page_key_code)
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
    all_article = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {}
        start = 1
        for i in range(start, Total_lenght+1):
            page_key = page_key_code + str(i)
            all_link_page = All_links_data[page_key]["Links"]
            for url in all_link_page:
                future = executor.submit(scrape_article, url, page_key_code)
                future_to_url[future] = (page_key, url)
        
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
            page_key, url = future_to_url[future]
            try:
                article_content = future.result()
                if article_content:
                    article_key = f"{page_key}_tbmgar_Article_{len([k for k in all_article if k.startswith(page_key)]) + 1}"
                    all_article[article_key] = article_content
            except Exception as e:
                print(f"Error processing {url}: {str(e)}")
    
    Failure_count = sum(1 for article in all_article.values() if article["Response"] != 200)
    print(f"Total Failure in the {page_key_list[1]} article: {Failure_count}")
    
    save_file_name = f"tbmgar_ALL_content_{page_key_list[1]}.json"
    print(save_file_name)
    path = "./data/parallel_content/"
    save_json(path, save_file_name, all_article)

def process_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            All_links_data = json.load(file)
            Total_lenght = len(All_links_data)
            print(f"Total page in {os.path.basename(file_path)}: {Total_lenght}")
        
            print(f"page key name: {list(All_links_data.keys())[-1]}")
            page_key_list = list(All_links_data.keys())[-1].split(" ")
            
            page_key_code = "Page "+page_key_list[1]+" "
            print(f"Page key code: {page_key_code}")
            get_content(All_links_data, Total_lenght, page_key_code, page_key_list)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {os.path.basename(file_path)}: {str(e)}")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {str(e)}")

def get_json_files(directory):
    json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))
    
    print(f"Processed {len(json_files)} files")


In [ ]:
%%time
# Example usage
directory_path = './data/links/'
get_json_files(directory_path)

Total page in tbmgar_ALL_link_བོད་ནང་གསར་འགྱུར།.json: 49
page key name: Page བོད་ནང་གསར་འགྱུར། 49
Page key code: Page བོད་ནང་གསར་འགྱུར། 
Total page in tbmgar_ALL_link_ཁོར་ཡུག་སྲུང་སྐྱོབ།.json: 3
page key name: Page ཁོར་ཡུག་སྲུང་སྐྱོབ། 3
Page key code: Page ཁོར་ཡུག་སྲུང་སྐྱོབ། 
Total page in tbmgar_ALL_link_བདམས་བཀོད་རྩོམ་ཡིག.json: 13
page key name: Page བདམས་བཀོད་རྩོམ་ཡིག 13
Page key code: Page བདམས་བཀོད་རྩོམ་ཡིག 
Total page in tbmgar_ALL_link_མགར་གྱི་མི་སྣ།.json: 1
page key name: Page མགར་གྱི་མི་སྣ། 1
Page key code: Page མགར་གྱི་མི་སྣ། 
Total page in tbmgar_ALL_link_རི་མོ།.json: 1
page key name: Page རི་མོ། 1
Page key code: Page རི་མོ། 
Total page in tbmgar_ALL_link_གཏམ་དཔེ།.json: 2
page key name: Page གཏམ་དཔེ། 2
Page key code: Page གཏམ་དཔེ། 
Total page in tbmgar_ALL_link_གནའ་གཏམ་དང་སྒྲུང་།.json: 3
page key name: Page གནའ་གཏམ་དང་སྒྲུང་། 3
Page key code: Page གནའ་གཏམ་དང་སྒྲུང་། 
Total page in tbmgar_ALL_link_ལེགས་བཤད།.json: 6
page key name: Page ལེགས་བཤད། 6
Page key code: Page ལེགས་བཤད

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]


  0%|          | 0/22 [00:00<?, ?it/s]




  0%|          | 0/25 [00:00<?, ?it/s]



  0%|          | 0/9 [00:00<?, ?it/s]





  0%|          | 0/56 [00:00<?, ?it/s]






  0%|          | 0/58 [00:00<?, ?it/s]







  0%|          | 0/127 [00:00<?, ?it/s]








  0%|          | 0/113 [00:00<?, ?it/s]









  0%|          | 0/489 [00:00<?, ?it/s]





  2%|▏         | 1/56 [00:06<05:31,  6.03s/it]




  4%|▍         | 1/25 [00:06<02:25,  6.05s/it]








  1%|          | 1/113 [00:06<11:14,  6.02s/it]


  5%|▍         | 1/22 [00:06<02:08,  6.14s/it]



 11%|█         | 1/9 [00:06<00:48,  6.05s/it]

  6%|▌         | 1/17 [00:06<01:38,  6.15s/it]







  1%|          | 1/127 [00:06<12:41,  6.04s/it]









  0%|          | 1/489 [00:06<49:09,  6.04s/it]






  2%|▏         | 1/58 [00:06<05:45,  6.07s/it]







  2%|▏         | 3/127 [00:06<03:19,  1.61s/it]








 10%|█         | 1/10 [00:06<0

Total Failure in the མགར་གྱི་མི་སྣ། article: 9
tbmgar_ALL_content_མགར་གྱི་མི་སྣ།.json
Successfully saved: tbmgar_ALL_content_མགར་གྱི་མི་སྣ།.json
Total page in tbmgar_ALL_link_གླུ་དབྱངས།.json: 1
page key name: Page གླུ་དབྱངས། 1
Page key code: Page གླུ་དབྱངས། 






0it [00:00, ?it/s]




0it [00:00, ?it/s]4/25 [02:04<00:23, 23.33s/it]


Total Failure in the གླུ་དབྱངས། article: 0
tbmgar_ALL_content_གླུ་དབྱངས།.json
Successfully saved: tbmgar_ALL_content_གླུ་དབྱངས།.json
Total page in tbmgar_ALL_link_བརྡ་སྤྲོད།  རྒྱུན་ཤེས།.json: 1
page key name: Page བརྡ་སྤྲོད།  རྒྱུན་ཤེས། 1
Page key code: Page བརྡ་སྤྲོད། 
Error processing file tbmgar_ALL_link_བརྡ་སྤྲོད།  རྒྱུན་ཤེས།.json: 'Page བརྡ་སྤྲོད། 1'
Total page in tbmgar_ALL_link_གླེང་སྟེགས་བཅར་འདྲི།.json: 1
page key name: Page གླེང་སྟེགས་བཅར་འདྲི། 1
Page key code: Page གླེང་སྟེགས་བཅར་འདྲི། 


...










  0%|          | 0/16 [00:00<?, ?it/s]









 38%|███▊      | 184/489 [08:16<30:31,  6.01s/it]


  4%|▍         | 4/90 [00:08<02:17,  1.60s/it]





  1%|          | 3/500 [00:08<19:25,  2.35s/it]  


  6%|▌         | 5/90 [00:08<01:48,  1.28s/it]





  9%|▉         | 1/11 [00:09<01:32,  9.25s/it]




 29%|██▉       | 151/519 [06:11<12:26,  2.03s/it]

 18%|█▊        | 2/11 [00:09<00:37,  4.15s/it]


 27%|██▋       | 3/11 [00:10<00:20,  2.52s/it]





  1%|▏         | 7/500 [00:11<10:15,  1.25s/it]

 88%|████████▊ | 7/8 [00:11<00:01,  1.34s/it]


 45%|████▌     | 5/11 [00:12<00:09,  1.59s/it]





  2%|▏         | 8/500 [00:12<09:04,  1.11s/it]

100%|██████████| 8/8 [00:12<00:00,  1.52s/it]


Total Failure in the འཛམ་གླིང་ཆོས་ལུགས། article: 8
tbmgar_ALL_content_འཛམ་གླིང་ཆོས་ལུགས།.json
Successfully saved: tbmgar_ALL_content_འཛམ་གླིང་ཆོས་ལུགས།.json
Total page in tbmgar_ALL_link_བརྡ་སྦྱོར་གླིང་།.json: 4
page key name: Page བརྡ་སྦྱོར་གླིང་། 4
Page key code: Page བརྡ་སྦྱོར་གླིང་། 




 55%|█████▍    | 6/11 [00:12<00:05,  1.17s/it]









 38%|███▊      | 185/489 [08:21<28:35,  5.64s/it]




 29%|██▉       | 152/519 [06:15<14:24,  2.35s/it]




 29%|██▉       | 153/519 [06:16<12:25,  2.04s/it]


  9%|▉         | 8/90 [00:13<02:19,  1.70s/it]


 64%|██████▎   | 7/11 [00:15<00:06,  1.67s/it]




 30%|██▉       | 154/519 [06:17<10:59,  1.81s/it]





  2%|▏         | 9/500 [00:15<14:07,  1.73s/it]


 11%|█         | 10/90 [00:16<01:54,  1.43s/it]








  6%|▋         | 1/16 [00:09<02:21,  9.44s/it]


 13%|█▎        | 12/90 [00:16<01:10,  1.10it/s]




 30%|██▉       | 155/519 [06:19<10:51,  1.79s/it]





  2%|▏         | 11/500 [00:16<10:06,  1.24s/it]






 16%|█▋        | 23/140 [01:02<14:09,  7.26s/it]


 14%|█▍        | 13/90 [00:18<01:14,  1.03it/s]





  2%|▏         | 12/500 [00:18<09:56,  1.22s/it]

  3%|▎         | 1/39 [00:05<03:42,  5.86s/it]








 19%|█▉        | 3/16 [00:11<00:39,  3.07s/it]





  3%|▎         | 14/500 [00:18<06:48,  1.19it/s]


Total Failure in the དུས་ཆེན་ངོ་སྤྲོད། article: 5
tbmgar_ALL_content_དུས་ཆེན་ངོ་སྤྲོད།.json
Successfully saved: tbmgar_ALL_content_དུས་ཆེན་ངོ་སྤྲོད།.json
Total page in tbmgar_ALL_link_རང་མོས།.json: 29
page key name: Page རང་མོས། 29
Page key code: Page རང་མོས། 










  0%|          | 0/289 [00:00<?, ?it/s]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.





 36%|███▌      | 185/519 [07:42<26:56,  4.84s/it]









 42%|████▏     | 203/489 [09:53<23:01,  4.83s/it]









 42%|████▏     | 205/489 [09:53<16:33,  3.50s/it]









 42%|████▏     | 206/489 [09:56<16:16,  3.45s/it]









 42%|████▏     | 207/489 [10:04<19:43,  4.20s/it]









 43%|████▎     | 208/489 [10:04<16:01,  3.42s/it]





 10%|█         | 52/500 [01:57<39:05,  5.24s/it]









 43%|████▎     | 209/489 [10:07<15:29,  3.32s/it]






 22%|██▏       | 31/140 [02:45<19:43, 10.86s/it]





 11%|█         | 53/500 [02:05<41:24,  5.56s/it]









 43%|████▎     | 210/489 [10:14<19:31,  4.20s/it]





 11%|█         | 54/500 [02:08<38:46,  5.22s/it]





 11%|█         | 55/500 [02:09<31:48,  4.29s/it]





 11%|█         | 56/500 [02:16<36:43,  4.96s/it]





 11%|█▏        | 57/500 [02:24<41:50,  5.67s/it]Some characters could n

In [ ]:
# 14:44

In [ ]:
json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
json_files